In [1]:
import logging

import logging
from collections import namedtuple
from collections import Counter
import random
from matplotlib import pyplot as plt

In [2]:
SEED = 42

In [3]:
Individual = namedtuple("Individual", ["genome", "fitness"])

def evaluate(state):
    cnt = Counter()
    cnt.update(sum((e for e in state), start=()))
    return len(cnt), -cnt.total()

In [4]:
import itertools

def tournament(population, tournament_size=2):
    # Take randomly 2 individuals from the  population and take the one with the higher fitness
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness)

def cross_over(g1, g2):

    if (len(g1) > len(g2)):
        len_small = len(g2)
    else:
        len_small = len(g1)
    
    cut2 = random.randint(1, len_small)
    cut1 = random.randint(cut2-1, len_small-1)

    return set(list(g1)[cut1:] + list(g2)[:cut2])

In [5]:

def mutation(solution):
    # We randomly modify our solution
    new_solution = set(solution)

    #print(solution)

    # We romove one random element from the solution 
    r = random.choice(list(new_solution))
    new_solution.remove(r)

    # We add one random element to the new solution
    # We take the element to add from the list of all possible elements in our original list of subsets
    # minus the elements already present in our current solution
    a = random.choice(list(all_lists - solution))
    new_solution.add(tuple(a))

    #print(new_solution)

    return new_solution

In [6]:
logging.getLogger().setLevel(logging.INFO)

In [7]:

NUM_GENERATIONS = 100

def evolution(population, POPULATION_SIZE, mutation_rate):

    OFFSPRING_SIZE = int(POPULATION_SIZE / 2)

    fitness_log = [(0, i.fitness) for i in population]

    for g in range(NUM_GENERATIONS):
        offspring = list()
        for i in range(OFFSPRING_SIZE):
            # Randomically perform a mutation on one gene
            if random.random() < mutation_rate:
                p = tournament(population)
                o = mutation(p.genome)
                #print(o)
            # Otherwise take two random individuals as parents and through crossover create 
            # a new individual --> offspring
            else:
                p1 = tournament(population)
                p2 = tournament(population)
                o = cross_over(p1.genome, p2.genome)
                #print(o)
            f = evaluate(o)
            fitness_log.append((g + 1, f))
            # take count of the list of all offspring created in one generation
            offspring.append(Individual(o, f))
        # After one generation add the offspring to the population
        population += offspring
        # Take from the enlarged new population only the best POPULATION_SIZE individuals
        population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]

    return population

In [8]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

def create_population(N):

    population = list()
    all_lists = set(tuple(_) for _ in problem(N, SEED))

    for genome in all_lists:

        gene = set()
        gene.add(genome)
        population.append(Individual(gene, evaluate(gene)))

    return population, all_lists, len(population)

In [9]:

for N in [5, 10, 20, 100, 
            500, 
            1000
            ]:
    mutation_rate = 0.55
    population, all_lists, population_size = create_population(N)
    solution = evolution(population, population_size, mutation_rate)

    w = (-solution[0][1][1])

    # print("Mutation rate: ", mutation_rate)
    logging.info(
        f" Solution for N={N:,}: "
        + f"w={w} "
        + f"(bloat={(w-N)/N*100:.0f}%)"
    )


INFO:root: Solution for N=5: w=5 (bloat=0%)
INFO:root: Solution for N=10: w=10 (bloat=0%)
INFO:root: Solution for N=20: w=24 (bloat=20%)
INFO:root: Solution for N=100: w=181 (bloat=81%)
INFO:root: Solution for N=500: w=1431 (bloat=186%)
INFO:root: Solution for N=1,000: w=3460 (bloat=246%)
